In [2]:
import numpy as np
import pandas as pd

In [3]:
from tkinter import *
import pandas

def parse_east(line):
    parts = line.split(",")
    parts0 = parts[0].split()
    parts1 = parts[1].split()
    if parts0[0].lower() == "east":
        return int(parts0[1])
    elif parts1[0].lower() == "east":
        return int(parts1[1])

def parse_west(line):
    parts = line.split(",")
    parts0 = parts[0].split()
    parts1 = parts[1].split()
    if parts0[0].lower() == "west":
        return int(parts0[1])
    elif parts1[0].lower() == "west":
        return int(parts1[1])

def s(L):
    return sum(L)
        
url="https://en.wikipedia.org/wiki/NBA_All-Star_Game"
table = pandas.read_html(url,header=0,index_col=0,parse_dates=True)
df = pandas.DataFrame(table[2])
df = df.dropna(how='any')

In [4]:
df_east = df.loc[:,"Result"]
df_east = df_east.apply(parse_east)
df_west = df.loc[:,"Result"]
df_west = df_west.apply(parse_west)
del(df["Result"])
del(df["Host arena"])
del(df["Game MVP"])
df.insert(0,"West",df_west)
df.insert(0,"East",df_east)
df = df.rename(index=lambda s: s.split("[")[0])
df = df.dropna(how='any')
df["Host city"] = df["Host city"].apply(lambda s: s.strip().split(",")[0])

df['Diff'] = abs(df['East'] - df['West'])
dfd = df.groupby('Diff').size().sort_values(ascending=False)
dfg = df.groupby('Host city').mean()

## 1.	What is the maximum and minimum score difference between all games?
maxdiff = max(dfg["Diff"])
print("Max difference:")
print(dfg.loc[dfg["Diff"] == maxdiff])
mindiff = min(dfg["Diff"])
print()
print("Min difference:")
print(dfg.loc[dfg["Diff"] == mindiff])

## 2.	If a city hosted a game more than once, what was the average score 
## for the Eastern Conference team and the Western Conference team? 
size = df.groupby('Host city').size()
dfg["Count"] = size
dfg = dfg.sort_values('Count')
dfg = dfg[dfg['Count'] > 1]
print()
print("Cities who hosted the game more than once:")
print(dfg)



Max difference:
             East  West  Diff
Host city                    
Cincinnati  137.0  94.0  43.0

Min difference:
             East   West  Diff
Host city                     
Milwaukee   124.0  125.0   1.0
San Diego   107.0  108.0   1.0
Washington  111.0  110.0   1.0

Cities who hosted the game more than once:
                     East        West       Diff  Count
Host city                                              
Atlanta        139.000000  140.000000   9.000000      2
Seattle        136.000000  144.000000   8.000000      2
Chicago        121.000000  108.500000  12.500000      2
Inglewood      121.000000  117.500000   5.500000      2
Orlando        131.000000  152.500000  21.500000      2
Denver         139.500000  130.000000   9.500000      2
Houston        131.333333  135.333333   5.333333      3
New Orleans    159.666667  158.333333   8.000000      3
St. Louis      128.000000  130.333333  11.000000      3
Los Angeles    130.000000  130.666667   5.333333      3
Phoeni